In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [3]:
def eq_diff_voiture(x,t,d,Db,Df,b,l,meq):
    p=0.055
    Cr=3.3e-4
    k=0.092
    theta_0=4.615
    req=1/(2/Db+2/Df)
    
    theta = x[0]
    theta_d = x[1]

    x_d = theta_d
    x_dd = 1/np.sin(theta)*(-theta_d**2*np.cos(theta)+np.sqrt(b**2+l**2-2*b*l*np.cos(theta))*(theta_d**2*b*l*np.sin(theta)**2/((b**2+l**2-2*b*l*np.cos(theta))**(3/2))+d/(meq*Db*b*l)*(-k*(theta-theta_0)*d*np.sqrt(b**2+l**2-2*b*l*np.cos(theta))/(Db*b*l*np.sin(theta))-Cr/req-p*Db*b*l/d*theta_d*np.sin(theta)/(np.sqrt(b**2+l**2-2*b*l*np.cos(theta))))))

    return np.array([x_d, x_dd])

In [5]:
def solve_eq_diff_voiture(params):
    return solve_ivp(eq_diff_voiture, [0, 1000], [0.1, 0], args=(params['d'], params['Db'], params['Df'], params['b'], params['l'], params['meq']), t_eval=np.linspace(0, 1000, 10000))

def calculate_speed_from_sol(sol, params):
    theta = sol.y[0]
    theta_d = sol.y[1]

    v_1 = params['Db']/params['d']*params['b']*params['l']*theta_d*np.sin(theta)/np.sqrt(params['b']**2+params['l']**2-2*params['b']*params['l']*np.cos(theta))
    # On calcule la suite sans le ressort avec une formule explicite
    v_fin = v_1[-1]
    req = 1/(2/params['Db']+2/params['Df'])
    v_2 = v_fin-3.3e-4/(params['mc']*req)*sol.t
    # On renvoie la vitesse en phase 1 concaténée avec la vitesse en phase 2
    return np.concatenate((v_1, v_2))

def plot_speed(sol, params):
    v = calculate_speed_from_sol(sol, params)
    plt.plot(sol.t, v)
    plt.xlabel('Temps (s)')
    plt.ylabel('Vitesse (m/s)')
    plt.show()